# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# Imports and Setup
import os
import json
import base64
from io import BytesIO
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from PIL import Image

In [ ]:
# Load environment variables
load_dotenv(override=True)

# API Keys
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

# Initialize clients
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

openai = OpenAI()
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)


# Model constants
MODELS = {
    "GPT-5": "gpt-5-mini",
    "Claude Sonnet": "claude-sonnet-4-5-20250929", 
    "Gemini": "gemini-2.5-flash"
}

print("All API clients initialized successfully!")


In [ ]:
def main_claude(messages, tools):

    response = anthropic.chat.completions.create(
        model=MODELS["Claude Sonnet"],
        messages=messages,
        tools=tools
    )
    
    return response

def main_gpt(messages, tools):

    response = openai.chat.completions.create(
        model=MODELS["GPT-5"],
        messages=messages,
        tools=tools
    )
    
    return response

def main_gemini(messages, tools):

    response = gemini.chat.completions.create(
        model=MODELS["Gemini"],
        messages=messages,
        tools=tools
    )
    
    return response

In [ ]:
def main_model(messages, model, tools):
    if model=="GPT-5":
        result = main_gpt(messages, tools)
    elif model=="Claude Sonnet":
        result = main_claude(messages, tools)
    elif model=="Gemini":
        result = main_gemini(messages, tools)        
    else:
        raise ValueError("Unknown model")
    
    return result

In [ ]:
initial_system_prompt = """You are an individual that possesses a unique
and highly valuable combination of deep technical expertise and 
excellent communication skills. You grasp complex, specialized 
concepts and then distill them into simple, understandable terms 
for people without the same technical background.

IMPORTANT: When users ask for code examples or programming concepts, 
you MUST use the generate_code_example tool to provide working code examples. 
Do not write code directly in your response - always call the tool first.

Present your answer as markdown and use the tool to generate working code examples 
when relevant to help illustrate the concepts."""

# Different system prompt for after tool calls
followup_system_prompt = """You are an individual that possesses a unique
and highly valuable combination of deep technical expertise and 
excellent communication skills. You grasp complex, specialized 
concepts and then distill them into simple, understandable terms 
for people without the same technical background.

IMPORTANT: You have already called the generate_code_example tool and received code. 
Now you should use that generated code in your response. Do NOT call the tool again.
Present your answer as markdown and incorporate the provided code examples 
to help illustrate the concepts."""


In [ ]:
# Code Example Generator Tool
def generate_code_example(language="python"):

    print(f"TOOL CALLED: Generating {language} code")
    
    return f"# Code example in {language}\n# This is a placeholder - implement your specific example here\n')"

# Tool definition
code_tool = {
    "name": "generate_code_example",
    "description": "Generate a working code example",
    "parameters": {
        "type": "object",
        "properties": {
            "language": {
                "type": "string", 
                "description": "The programming language (default: python)",
                "default": "python"
            }
        },
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": code_tool}]


In [ ]:

# Audio and Image Generation Functions
def transcribe_audio(audio_file):
    """Convert audio to text using OpenAI Whisper"""
    if audio_file is None:
        return None
    
    with open(audio_file, "rb") as audio:
        transcript = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio
        )
    return transcript.text

def text_to_speech(text):
    """Convert text to speech using OpenAI TTS"""
    response = openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input=text
    )
    return response.content

def generate_robot_image(code_snippet):
    """Generate an image of a robot developer at a monitor showing the code"""
    prompt = f"""A robot developer sitting at a computer monitor in a modern office. Robot is styled
    as Bender from Futurama.
    The monitor displays code on the screen showing: {code_snippet[:100]}...
    The robot has a friendly, helpful appearance with glowing eyes and mechanical hands typing.
    The scene is well-lit with a professional coding environment background."""
    
    response = openai.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        n=1,
        response_format="b64_json"
    )
    
    image_base64 = response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))


In [ ]:
def handle_tool_calls(message):
    responses = []
    generated_code = None
    
    for tool_call in message.tool_calls:
        if tool_call.function.name == "generate_code_example":
            arguments = json.loads(tool_call.function.arguments)
            language = arguments.get('language', 'python')
            code_example = generate_code_example(language)
            generated_code = code_example
            
            responses.append({
                "role": "tool",
                "content": code_example,
                "tool_call_id": tool_call.id
            })
    
    return responses, generated_code

def chat(message, history, model_choice):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": initial_system_prompt}] + history + [{"role": "user", "content": message}]
    
    generated_code = None
    image = None
    
    response = main_model(
        model=model_choice,
        messages=messages,
        tools=tools
    )

    print(f"Response finish_reason: {response.choices[0].finish_reason}")
    print(f"Response tool_calls: {response.choices[0].message.tool_calls}")

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses, code = handle_tool_calls(message)
        generated_code = code
        messages.append(message)
        messages.extend(responses)
        response = main_model(
            model=model_choice,
            messages=messages,
            tools=tools
        )
        messages[0] = {"role": "system", "content": followup_system_prompt}

        response = main_model(
            model=model_choice,
            messages=messages,
            tools=tools
        )

        print(f"After tool call - finish_reason: {response.choices[0].finish_reason}")
    
    reply = response.choices[0].message.content
        
    if generated_code:
        try:
            print(f"Generating image for code: {generated_code}")
            image = generate_robot_image(generated_code)
        except Exception as e:
            print(f"Image generation failed: {e}")
    
    try:
        audio = text_to_speech(reply)
    except Exception as e:
        print(f"Audio generation failed: {e}")
        audio = None
    
    return reply, audio, image

In [ ]:
def process_text_input(message, history, model_choice):
    if not message.strip():
        return history, None, None
    
    reply, audio, image = chat(message, history, model_choice)
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": reply})

    return history, audio, image

def process_audio_input(audio_file, history, model_choice):
    if audio_file is None:
        return history, None, None
    
    message = transcribe_audio(audio_file)
    if not message:
        return history, None, None
    
    return process_text_input(message, history, model_choice)

with gr.Blocks(title="Technical Q&A Assistant") as demo:
    gr.Markdown("# 🤖 Technical Q&A Assistant")
    gr.Markdown("Ask technical questions and get explanations with code examples!")
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                height=400,
                type="messages",
                label="Conversation"
            )
            
            with gr.Row():
                text_input = gr.Textbox(
                    placeholder="Ask a technical question...",
                    label="Text Input",
                    scale=4
                )
                text_submit = gr.Button("Send", scale=1)
            
            with gr.Row():
                audio_input = gr.Audio(
                    sources=["microphone"],
                    type="filepath",
                    label="Voice Input"
                )
                audio_submit = gr.Button("Send Voice", scale=1)
        
        with gr.Column(scale=1):
            model_selector = gr.Dropdown(
                choices=list(MODELS.keys()),
                value="GPT-5",
                label="Choose Model"
            )
            
            audio_output = gr.Audio(
                label="Audio Response",
                autoplay=True
            )
            
            image_output = gr.Image(
                label="Generated Image",
                height=300,
                interactive=False
            )
    
    text_submit.click(
        process_text_input,
        inputs=[text_input, chatbot, model_selector],
        outputs=[chatbot, audio_output, image_output]
    ).then(
        lambda: "",
        outputs=text_input
    )
    
    audio_submit.click(
        process_audio_input,
        inputs=[audio_input, chatbot, model_selector],
        outputs=[chatbot, audio_output, image_output]
    )
    
    text_input.submit(
        process_text_input,
        inputs=[text_input, chatbot, model_selector],
        outputs=[chatbot, audio_output, image_output]
    ).then(
        lambda: "",  
        outputs=text_input
    )

demo.launch()
